### This Notebook finds the values in all counties from a gridded data. A sample of 100 latitude and longitude pair is taken from the shapefile of a county. The values of the variables from the netcdf are then interpolated from the grid to this set of latitude and longitude.

In [1]:
## conda environment Weather_Prediction

import os
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr

In [2]:
data = xr.open_dataset("data.nc") ## one month average of the variables
data

<xarray.Dataset> Size: 7MB
Dimensions:        (valid_time: 12, latitude: 35, longitude: 81)
Coordinates:
  * valid_time     (valid_time) datetime64[ns] 96B 2010-01-01 ... 2010-12-01
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
Data variables: (12/48)
    d2m            (valid_time, latitude, longitude) float32 136kB ...
    t2m            (valid_time, latitude, longitude) float32 136kB ...
    bcaod550       (valid_time, latitude, longitude) float32 136kB ...
    chnk           (valid_time, latitude, longitude) float32 136kB ...
    duaod550       (valid_time, latitude, longitude) float32 136kB ...
    lai_hv         (valid_time, latitude, longitude) float32 136kB ...
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 136kB ...
    aermsssss      (valid_time, latitude, longitude) float32 136kB ...
    aermssssm      (valid_time, latitude, longitude) float32 136kB ...
    aermssssl      (valid_time, latitude, longitude) float32 136kB ...
    aermsssu       (valid_time, latitude, longitude) float32 136kB ...
    aermssso2      (valid_time, latitude, longitude) float32 136kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
## resample the data to 1 year average

year_avg = data.resample(valid_time='1YE').mean(dim='valid_time')

## delete variables that have NaN in them

vars_with_nan = [var for var in year_avg if year_avg[var].isnull().any()]
year_avg= year_avg.drop_vars(vars_with_nan)
year_avg

<xarray.Dataset> Size: 523kB
Dimensions:        (valid_time: 1, latitude: 35, longitude: 81)
Coordinates:
  * latitude       (latitude) float64 280B 49.5 48.75 48.0 ... 25.5 24.75 24.0
  * longitude      (longitude) float64 648B -126.0 -125.2 ... -66.75 -66.0
  * valid_time     (valid_time) datetime64[ns] 8B 2010-12-31
Data variables: (12/46)
    d2m            (valid_time, latitude, longitude) float32 11kB 278.3 ... 2...
    t2m            (valid_time, latitude, longitude) float32 11kB 281.6 ... 2...
    bcaod550       (valid_time, latitude, longitude) float32 11kB 0.006512 .....
    chnk           (valid_time, latitude, longitude) float32 11kB 0.01415 ......
    duaod550       (valid_time, latitude, longitude) float32 11kB 0.001253 .....
    lai_hv         (valid_time, latitude, longitude) float32 11kB 3.752 ... 0.0
    ...             ...
    aermssomhphob  (valid_time, latitude, longitude) float32 11kB 9.168e-06 ....
    aermsssss      (valid_time, latitude, longitude) float32 11kB 6.539e-07 ....
    aermssssm      (valid_time, latitude, longitude) float32 11kB 4.983e-05 ....
    aermssssl      (valid_time, latitude, longitude) float32 11kB 1.505e-05 ....
    aermsssu       (valid_time, latitude, longitude) float32 11kB 2.849e-06 ....
    aermssso2      (valid_time, latitude, longitude) float32 11kB 6.057e-07 ....
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2024-10-10T00:09 GRIB to CDM+CF via cfgrib-0.9.1...

In [4]:
## note that even though the time dimension, essentially shows 2010-12-31,
## what it actually means is yearly average
year_avg.to_dataframe()

d2m         t2m  bcaod550      chnk  \
valid_time latitude longitude                                               
2010-12-31 49.5     -126.00    278.348907  281.556824  0.006512  0.014148   
                    -125.25    279.088837  282.437836  0.006916  0.013934   
                    -124.50    279.470001  282.923370  0.007213  0.013523   
                    -123.75    279.041290  282.562347  0.007478  0.013581   
                    -123.00    277.851837  281.525085  0.008017  0.014813   
...                                   ...         ...       ...       ...   
           24.0     -69.00     293.523407  297.542999  0.004872  0.012293   
                    -68.25     293.495728  297.488953  0.004856  0.012303   
                    -67.50     293.465454  297.455261  0.004833  0.012302   
                    -66.75     293.443970  297.427094  0.004824  0.012318   
                    -66.00     293.438751  297.397156  0.004825  0.012332   

                               duaod550    lai_hv    lai_lv            msl  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00    0.001253  3.752065  2.928701  101390.726562   
                    -125.25    0.001246  4.228424  1.811178  101402.250000   
                    -124.50    0.001238  2.194234  0.692861  101427.570312   
                    -123.75    0.001204  0.905955  0.624898  101459.429688   
                    -123.00    0.001140  3.628265  2.724324  101485.656250   
...                                 ...       ...       ...            ...   
           24.0     -69.00     0.024328  0.000000  0.000000  101571.789062   
                    -68.25     0.024477  0.000000  0.000000  101578.000000   
                    -67.50     0.024507  0.000000  0.000000  101583.164062   
                    -66.75     0.024585  0.000000  0.000000  101590.195312   
                    -66.00     0.024893  0.000000  0.000000  101595.625000   

                               omaod550         pm2p5  ...     aermssdul  \
valid_time latitude longitude                          ...                 
2010-12-31 49.5     -126.00    0.055563  5.288607e-09  ...  5.697568e-07   
                    -125.25    0.060980  6.618710e-09  ...  5.552510e-07   
                    -124.50    0.063776  7.792362e-09  ...  5.443322e-07   
                    -123.75    0.067136  1.080445e-08  ...  5.231616e-07   
                    -123.00    0.076543  1.931933e-08  ...  4.871432e-07   
...                                 ...           ...  ...           ...   
           24.0     -69.00     0.046011  9.519893e-09  ...  1.086019e-05   
                    -68.25     0.045661  9.399162e-09  ...  1.102058e-05   
                    -67.50     0.045540  9.288230e-09  ...  1.103588e-05   
                    -66.75     0.045691  9.213097e-09  ...  1.106749e-05   
                    -66.00     0.045851  9.209978e-09  ...  1.116341e-05   

                               aermssbchphil  aermssomhphil  aermssbchphob  \
valid_time latitude longitude                                                
2010-12-31 49.5     -126.00     1.359443e-07   2.107467e-06   5.559153e-07   
                    -125.25     1.506114e-07   2.379295e-06   5.843853e-07   
                    -124.50     1.616488e-07   2.564180e-06   6.047914e-07   
                    -123.75     1.798799e-07   2.806439e-06   6.146706e-07   
                    -123.00     2.252995e-07   3.389544e-06   6.264335e-07   
...                                      ...            ...            ...   
           24.0     -69.00      3.103107e-08   5.009322e-07   4.866120e-07   
                    -68.25      3.011268e-08   4.867323e-07   4.857583e-07   
                    -67.50      2.886006e-08   4.733052e-07   4.846137e-07   
                    -66.75      2.775320e-08   4.601205e-07   4.847639e-07   
                    -66.00      2.699588e-08   4.502798e-07   4.856791e-07   

                 

In [5]:
## Lets load the county shapefile

SHAPE_PATH = os.path.join("County_shapefile",'gz_2010_us_050_00_500k.shp')
county_gdf = gpd.read_file(SHAPE_PATH)

county_gdf

,GEO_ID,STATE,COUNTY,NAME,LSAD,CENSUSAREA,geometry
0,0500000US01029,01,029,Cleburne,County,560.100,"POLYGON ((-85.38872 33.91304, -85.38088 33.873..."
1,0500000US01031,01,031,Coffee,County,678.972,"POLYGON ((-86.03044 31.61894, -86.00408 31.619..."
2,0500000US01037,01,037,Coosa,County,650.926,"POLYGON ((-86.00928 33.10164, -86.00917 33.090..."
3,0500000US01039,01,039,Covington,County,1030.456,"POLYGON ((-86.34851 30.99434, -86.35023 30.994..."
4,0500000US01041,01,041,Crenshaw,County,608.840,"POLYGON ((-86.14699 31.68045, -86.14711 31.663..."
...,...,...,...,...,...,...,...
3216,0500000US72131,72,131,San Sebastián,Muno,70.423,"POLYGON ((-66.90748 18.25314, -66.90739 18.253..."
3217,0500000US72133,72,133,Santa Isabel,Muno,34.023,"POLYGON ((-66.37968 17.94398, -66.38029 17.943..."
3218,0500000US72137,72,137,Toa Baja,Muno,23.241,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ..."
3219,0500000US72139,72,139,Trujillo Alto,Muno,20.764,"POLYGON ((-66.02917 18.37590, -66.02828 18.376..."


In [6]:
geometry_column=county_gdf['geometry']
geometry_column

0       POLYGON ((-85.38872 33.91304, -85.38088 33.873...
1       POLYGON ((-86.03044 31.61894, -86.00408 31.619...
2       POLYGON ((-86.00928 33.10164, -86.00917 33.090...
3       POLYGON ((-86.34851 30.99434, -86.35023 30.994...
4       POLYGON ((-86.14699 31.68045, -86.14711 31.663...
                              ...                        
3216    POLYGON ((-66.90748 18.25314, -66.90739 18.253...
3217    POLYGON ((-66.37968 17.94398, -66.38029 17.943...
3218    MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...
3219    POLYGON ((-66.02917 18.37590, -66.02828 18.376...
3220    POLYGON ((-66.85229 17.95500, -66.85280 17.955...
Name: geometry, Length: 3221, dtype: geometry

In [7]:
type(geometry_column)

geopandas.geoseries.GeoSeries

In [8]:
lat_lon=geometry_column.get_coordinates(index_parts=True)
lat_lon

x          y
0    0   -85.388717  33.913044
     1   -85.380885  33.873508
     2   -85.379455  33.866291
     3   -85.377426  33.856047
     4   -85.376403  33.850656
...             ...        ...
3220 202 -66.833718  17.989763
     203 -66.835282  17.988274
     204 -66.835429  17.986323
     205 -66.836682  17.965971
     206 -66.852288  17.955004

[837022 rows x 2 columns]

In [9]:
def single_county(var):
    
    ''' Function to interpolate the values in a single county by taking a sample of 10 latitudes and
         10 longitudes. The interpolation done by xarray will give a set of 100 points.  
        Args:
        --------
             var (int): The index of the county in the shapefile.

        Returns:
        --------
            summary_df: Dataframe consisting the values of the variables interpolated in the county.
    '''    
    
    
    longitude= lat_lon.loc[(var), 'x']  # get the longitude
    longitude=longitude.sample(n=10)    # randomly select 10 points

    latitude= lat_lon.loc[(var), 'y']  # get the latitude
    latitude=latitude.sample(n=10)    ## randomly select 10 points

    lat_list=latitude.tolist()
    lon_list=longitude.tolist()

    year_avg_finer= year_avg.interp(longitude=lon_list, latitude=lat_list)

    summary = year_avg_finer.groupby("valid_time").mean(["latitude", "longitude"])
    summary_df = summary.to_dataframe()

    return summary_df

In [10]:
%%time

## get the average for all of the counties

df_list=[]

for i in range(0,3221): ## A total of 3221 counties. index in shape file starts from 0 and ends in 3220
    try:
        df_list.append(single_county(i))
    except:
        pass

CPU times: total: 2min 13s
Wall time: 2min 16s


In [11]:
initial_df=pd.concat(df_list)
initial_df

,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,msl,omaod550,pm2p5,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
valid_time,,,,,,,,,,,,,,,,,,,,,
2010-12-31,282.597941,289.790877,0.006230,0.018000,0.007958,3.555540,2.739758,101708.396641,0.077332,1.675008e-08,...,0.000002,1.377764e-07,0.000003,5.241716e-07,0.000013,2.002221e-07,0.000013,7.380544e-07,0.000009,3.704147e-06
2010-12-31,284.517263,291.516955,0.006495,0.017539,0.011870,3.909951,2.431012,101689.757734,0.073600,1.504295e-08,...,0.000004,1.414761e-07,0.000002,5.485955e-07,0.000013,2.563247e-07,0.000018,1.347213e-06,0.000008,9.576379e-07
2010-12-31,283.251477,290.603399,0.006225,0.018000,0.009715,3.842662,2.582652,101699.504013,0.076119,1.527813e-08,...,0.000003,1.302037e-07,0.000003,5.311232e-07,0.000013,2.188952e-07,0.000015,9.003240e-07,0.000008,2.437844e-06
2010-12-31,284.524885,291.644218,0.006595,0.017658,0.012248,3.895775,2.645750,101690.138268,0.074700,1.566281e-08,...,0.000004,1.455114e-07,0.000002,5.551415e-07,0.000013,2.592503e-07,0.000018,1.385016e-06,0.000008,9.672082e-07
2010-12-31,284.038968,291.423411,0.006455,0.017982,0.011712,3.999986,2.493327,101691.131354,0.074308,1.497188e-08,...,0.000004,1.383212e-07,0.000002,5.474610e-07,0.000013,2.505408e-07,0.000018,1.261081e-06,0.000008,1.018333e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2010-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2010-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Note that there are NaN values because the data, 'data.nc' is for the Continental USA, whereas the county shapefile consists of other locations as well such as Alaska, hawaii, guam, puertorico etc.

In [12]:
final_df=initial_df.reset_index(drop=True) ## reset index and dont include the valid_time as a column
final_df

,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,msl,omaod550,pm2p5,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,282.597941,289.790877,0.006230,0.018000,0.007958,3.555540,2.739758,101708.396641,0.077332,1.675008e-08,...,0.000002,1.377764e-07,0.000003,5.241716e-07,0.000013,2.002221e-07,0.000013,7.380544e-07,0.000009,3.704147e-06
1,284.517263,291.516955,0.006495,0.017539,0.011870,3.909951,2.431012,101689.757734,0.073600,1.504295e-08,...,0.000004,1.414761e-07,0.000002,5.485955e-07,0.000013,2.563247e-07,0.000018,1.347213e-06,0.000008,9.576379e-07
2,283.251477,290.603399,0.006225,0.018000,0.009715,3.842662,2.582652,101699.504013,0.076119,1.527813e-08,...,0.000003,1.302037e-07,0.000003,5.311232e-07,0.000013,2.188952e-07,0.000015,9.003240e-07,0.000008,2.437844e-06
3,284.524885,291.644218,0.006595,0.017658,0.012248,3.895775,2.645750,101690.138268,0.074700,1.566281e-08,...,0.000004,1.455114e-07,0.000002,5.551415e-07,0.000013,2.592503e-07,0.000018,1.385016e-06,0.000008,9.672082e-07
4,284.038968,291.423411,0.006455,0.017982,0.011712,3.999986,2.493327,101691.131354,0.074308,1.497188e-08,...,0.000004,1.383212e-07,0.000002,5.474610e-07,0.000013,2.505408e-07,0.000018,1.261081e-06,0.000008,1.018333e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3219,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## We will now need to merge the dataframe with the original shapefile.

In [13]:
## lets first create a fips column in the original shape file

## lets create the fips column in the dataframe which is the sum of state and county both as dtype string.
## Note: not a numerical sum
## This step is necessary because there can be several counties with idential names. 

county_gdf['fips']=county_gdf['STATE'] + county_gdf['COUNTY']
county_gdf.dtypes
## convert the data type
county_gdf['fips']=county_gdf['fips'].astype(str).astype(int)
county_gdf.dtypes

GEO_ID          object
STATE           object
COUNTY          object
NAME            object
LSAD            object
CENSUSAREA     float64
geometry      geometry
fips             int32
dtype: object

In [14]:
county_gdf = county_gdf.drop([ 'GEO_ID','CENSUSAREA','STATE','COUNTY','LSAD'], axis=1)
county_gdf

,NAME,geometry,fips
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041
...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139


In [15]:
type(county_gdf)

geopandas.geodataframe.GeoDataFrame

In [16]:
merged_df=pd.merge(county_gdf, final_df, left_index=True, right_index=True)
merged_df

,NAME,geometry,fips,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,282.597941,289.790877,0.006230,0.018000,0.007958,3.555540,2.739758,...,0.000002,1.377764e-07,0.000003,5.241716e-07,0.000013,2.002221e-07,0.000013,7.380544e-07,0.000009,3.704147e-06
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031,284.517263,291.516955,0.006495,0.017539,0.011870,3.909951,2.431012,...,0.000004,1.414761e-07,0.000002,5.485955e-07,0.000013,2.563247e-07,0.000018,1.347213e-06,0.000008,9.576379e-07
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037,283.251477,290.603399,0.006225,0.018000,0.009715,3.842662,2.582652,...,0.000003,1.302037e-07,0.000003,5.311232e-07,0.000013,2.188952e-07,0.000015,9.003240e-07,0.000008,2.437844e-06
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039,284.524885,291.644218,0.006595,0.017658,0.012248,3.895775,2.645750,...,0.000004,1.455114e-07,0.000002,5.551415e-07,0.000013,2.592503e-07,0.000018,1.385016e-06,0.000008,9.672082e-07
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041,284.038968,291.423411,0.006455,0.017982,0.011712,3.999986,2.493327,...,0.000004,1.383212e-07,0.000002,5.474610e-07,0.000013,2.505408e-07,0.000018,1.261081e-06,0.000008,1.018333e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3216,San Sebastián,"POLYGON ((-66.90748 18.25314, -66.90739 18.253...",72131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3217,Santa Isabel,"POLYGON ((-66.37968 17.94398, -66.38029 17.943...",72133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3218,Toa Baja,"MULTIPOLYGON (((-66.13957 18.46232, -66.13956 ...",72137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3219,Trujillo Alto,"POLYGON ((-66.02917 18.37590, -66.02828 18.376...",72139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
county_df=merged_df.dropna()
county_df.head()

,NAME,geometry,fips,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,"POLYGON ((-85.38872 33.91304, -85.38088 33.873...",1029,282.597941,289.790877,0.006230,0.018000,0.007958,3.555540,2.739758,...,0.000002,1.377764e-07,0.000003,5.241716e-07,0.000013,2.002221e-07,0.000013,7.380544e-07,0.000009,3.704147e-06
1,Coffee,"POLYGON ((-86.03044 31.61894, -86.00408 31.619...",1031,284.517263,291.516955,0.006495,0.017539,0.011870,3.909951,2.431012,...,0.000004,1.414761e-07,0.000002,5.485955e-07,0.000013,2.563247e-07,0.000018,1.347213e-06,0.000008,9.576379e-07
2,Coosa,"POLYGON ((-86.00928 33.10164, -86.00917 33.090...",1037,283.251477,290.603399,0.006225,0.018000,0.009715,3.842662,2.582652,...,0.000003,1.302037e-07,0.000003,5.311232e-07,0.000013,2.188952e-07,0.000015,9.003240e-07,0.000008,2.437844e-06
3,Covington,"POLYGON ((-86.34851 30.99434, -86.35023 30.994...",1039,284.524885,291.644218,0.006595,0.017658,0.012248,3.895775,2.645750,...,0.000004,1.455114e-07,0.000002,5.551415e-07,0.000013,2.592503e-07,0.000018,1.385016e-06,0.000008,9.672082e-07
4,Crenshaw,"POLYGON ((-86.14699 31.68045, -86.14711 31.663...",1041,284.038968,291.423411,0.006455,0.017982,0.011712,3.999986,2.493327,...,0.000004,1.383212e-07,0.000002,5.474610e-07,0.000013,2.505408e-07,0.000018,1.261081e-06,0.000008,1.018333e-06


In [18]:
type(county_df)

geopandas.geodataframe.GeoDataFrame

In [19]:
## convert into pandas dataframe without the geometry column

county_var=pd.DataFrame(county_df.drop(columns='geometry')) 
county_var

,NAME,fips,d2m,t2m,bcaod550,chnk,duaod550,lai_hv,lai_lv,msl,...,aermssdul,aermssbchphil,aermssomhphil,aermssbchphob,aermssomhphob,aermsssss,aermssssm,aermssssl,aermsssu,aermssso2
0,Cleburne,1029,282.597941,289.790877,0.006230,0.018000,0.007958,3.555540,2.739758,101708.396641,...,0.000002,1.377764e-07,0.000003,5.241716e-07,0.000013,2.002221e-07,0.000013,7.380544e-07,0.000009,3.704147e-06
1,Coffee,1031,284.517263,291.516955,0.006495,0.017539,0.011870,3.909951,2.431012,101689.757734,...,0.000004,1.414761e-07,0.000002,5.485955e-07,0.000013,2.563247e-07,0.000018,1.347213e-06,0.000008,9.576379e-07
2,Coosa,1037,283.251477,290.603399,0.006225,0.018000,0.009715,3.842662,2.582652,101699.504013,...,0.000003,1.302037e-07,0.000003,5.311232e-07,0.000013,2.188952e-07,0.000015,9.003240e-07,0.000008,2.437844e-06
3,Covington,1039,284.524885,291.644218,0.006595,0.017658,0.012248,3.895775,2.645750,101690.138268,...,0.000004,1.455114e-07,0.000002,5.551415e-07,0.000013,2.592503e-07,0.000018,1.385016e-06,0.000008,9.672082e-07
4,Crenshaw,1041,284.038968,291.423411,0.006455,0.017982,0.011712,3.999986,2.493327,101691.131354,...,0.000004,1.383212e-07,0.000002,5.474610e-07,0.000013,2.505408e-07,0.000018,1.261081e-06,0.000008,1.018333e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,Niobrara,56027,271.620593,282.302309,0.005395,0.018000,0.007585,3.394275,0.729611,101401.029651,...,0.000008,1.081958e-07,0.000002,4.649381e-07,0.000010,2.122870e-07,0.000013,8.382868e-07,0.000004,1.010639e-06
3139,Platte,56031,270.659080,281.302781,0.005111,0.018000,0.006658,4.002949,0.769710,101411.131290,...,0.000007,1.015922e-07,0.000002,4.414300e-07,0.000009,2.035717e-07,0.000013,7.850591e-07,0.000004,1.105991e-06
3140,Sweetwater,56037,268.699994,278.306938,0.004731,0.018000,0.004273,2.963378,0.521521,101612.298364,...,0.000004,9.234859e-08,0.000002,4.103250e-07,0.000008,2.020761e-07,0.000012,8.884089e-07,0.000003,1.334658e-06
3141,Washakie,56043,269.946087,279.434588,0.005484,0.018000,0.003667,4.857899,0.494686,101430.695846,...,0.000003,1.018511e-07,0.000002,4.805101e-07,0.000010,2.185095e-07,0.000013,9.747660e-07,0.000003,6.134079e-07


In [20]:
county_var.to_pickle('county_variables.pkl')